In [4]:
import requests
import json
from pprint import pprint
import numpy as np
import pandas as pd
import os
import csv
import datetime as dt

from sqlalchemy import MetaData
from sqlalchemy import Table
from sqlalchemy import Column
from sqlalchemy import Integer, String, Float

import sqlalchemy
from sqlalchemy import create_engine, func, inspect
from sqlalchemy_utils import database_exists, create_database
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, Date

from math import radians, sin, cos, asin, sqrt # used to calculate distance of room and city longs and lats

from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

# Google API Key
# from config import gkey
from flask import Flask, jsonify

In [6]:
results = requests.get('https://public.opendatasoft.com/api/records/1.0/download/?dataset=air-bnb-listings&rows=100&format=json').json()


In [7]:
#pprint(results)

In [8]:
results = [r["fields"] for r in results]
#results

In [9]:
results_df = pd.DataFrame(results)
results_df.head()

,availability_365,calculated_host_listings_count,city,column_10,column_19,column_20,coordinates,host_id,id,last_review,minimum_nights,name,neighbourhood,number_of_reviews,reviews_per_month,room_type,updated_date
0,292,7,Lisbon,125,Portugal,"Portugal, Lisbon, Avenidas Novas","[38.74584282796673, -9.152359763007405]",15282851,2998105,2019-04-17,2,Exciting Lisbon Entrecampos,Avenidas Novas,297,4.97,Entire home/apt,2019-04-22
1,0,1,Lisbon,500,Portugal,"Portugal, Lisbon, Alcntara","[38.70213919994125, -9.177363170810427]",15307946,3003712,NaN,1,Fantastic Apartment with deck 120m2,Alcntara,0,NaN,Private room,2019-04-22
2,0,1,Lisbon,650,Portugal,"Portugal, Lisbon, So Domingos de Benfica","[38.74499852902329, -9.168202490560926]",15330291,3008031,NaN,1,Champions League apartment,So Domingos de Benfica,0,NaN,Entire home/apt,2019-04-22
3,0,1,Lisbon,500,Portugal,"Portugal, Lisbon, Algs, Linda-a-Velha e Cruz Q...","[38.70144477298908, -9.240587547103432]",15415294,3025693,NaN,1,Quarto em Algés,"Algs, Linda-a-Velha e Cruz Quebrada-Dafundo",0,NaN,Private room,2019-04-22
4,277,3,Lisbon,37,Portugal,"Portugal, Lisbon, Misericrdia","[38.71109888002856, -9.148347820132365]",15580486,3063405,2019-04-19,2,Camões Lounge Apartment w/Courtyard,Misericrdia,299,5.02,Entire home/apt,2019-04-22


In [13]:
cleanData = results_df[[ 'column_10','id', 'coordinates', 'room_type', "city", 'column_19']]
cleanData.head()

,column_10,id,coordinates,room_type,city,column_19
0,125,2998105,"[38.74584282796673, -9.152359763007405]",Entire home/apt,Lisbon,Portugal
1,500,3003712,"[38.70213919994125, -9.177363170810427]",Private room,Lisbon,Portugal
2,650,3008031,"[38.74499852902329, -9.168202490560926]",Entire home/apt,Lisbon,Portugal
3,500,3025693,"[38.70144477298908, -9.240587547103432]",Private room,Lisbon,Portugal
4,37,3063405,"[38.71109888002856, -9.148347820132365]",Entire home/apt,Lisbon,Portugal


In [14]:
cleanData = cleanData.rename(index=str, columns={"column_10":"Price",
                                                 "id":"Room_Id", 
                                                 "coordinates":"Coordinates",
                                                 "column_19":"Country", 
                                                 "room_type":"Room_Type",
                                                 "city":"City"})

cleanData.head()

,Price,Room_Id,Coordinates,Room_Type,City,Country
0,125,2998105,"[38.74584282796673, -9.152359763007405]",Entire home/apt,Lisbon,Portugal
1,500,3003712,"[38.70213919994125, -9.177363170810427]",Private room,Lisbon,Portugal
2,650,3008031,"[38.74499852902329, -9.168202490560926]",Entire home/apt,Lisbon,Portugal
3,500,3025693,"[38.70144477298908, -9.240587547103432]",Private room,Lisbon,Portugal
4,37,3063405,"[38.71109888002856, -9.148347820132365]",Entire home/apt,Lisbon,Portugal


In [15]:
# Create two lists for the loop results to be placed
lat = []
lon = []

# For each row in a varible,
for row in cleanData['Coordinates']:
   # Try to,
   try:
       # Split the row by comma and append
       # everything before the comma to lat
       lat.append(row[0])
       # Split the row by comma and append
       # everything after the comma to lon
       lon.append(row[1])
   # But if you get an error
   except:
       # append a missing value to lat
       lat.append(np.NaN)
       # append a missing value to lon
       lon.append(np.NaN)

# Create two new columns from lat and lon
cleanData['Room_Lat'] = lat
cleanData['Room_Long'] = lon


In [16]:
# cleanData = cleanData.drop(columns=["Coordinates"])
# cleanData.head()

In [17]:
cleanData["Selection"] = cleanData["City"]+", "+cleanData["Country"]
# cleanData.head(5)

In [10]:
cleanData['City'] = cleanData['City'].str.replace('-',' ')

In [11]:
rooms_lat = []   # create empty lists
rooms_long = []
cities_lat = [] 
cities_long = []
calculated_distance = []

for index, row in cleanData.iterrows():
    room_lat1 = row['Room_Lat']
    room_long1 = row['Room_Long']
    
# Build the endpoint URL
    gkey = "<KEY>"
    target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
    'address={0}&key={1}').format(row, gkey)

#Run a request to endpoint and convert result to json
    cities_lat_long = requests.get(target_url).json()

    city_lat2 = cities_lat_long["results"][0]["geometry"]["location"]["lat"]
    city_long2 = cities_lat_long["results"][0]["geometry"]["location"]["lng"]
    
#CALCULATES DISTANCE OF ROOMS AND CITIES LAT AND LONGS
    def distance(room_long1, room_lat1, city_long2, city_lat2):
        room_long1, room_lat1, city_long2, city_lat2 = map(radians, [room_long1, room_lat1, city_long2, city_lat2])
        dlon = city_long2 - room_long1
        dlat = city_lat2 - room_lat1
        a = sin(dlat / 2) ** 2 + cos(room_lat1) * cos(city_lat2) * sin(dlon / 2) ** 2
        return 2 * 3958.756 * asin(sqrt(a)) #6371 km / 3958.756 for miles

    room_city_distance = distance(room_long1, room_lat1, city_long2, city_lat2)

    rooms_lat.append(room_lat1)
    rooms_long.append(room_long1)
    cities_lat.append(city_lat2)
    cities_long.append(city_long2)
    calculated_distance.append(room_city_distance)
    
print("rooms_lat:", rooms_lat)
print("rooms_long:", rooms_long)
print("cities_lat:", cities_lat)
print("cities_long:", cities_long)
print("calculated_distance:", calculated_distance)

# this site confirmed the calculation is correct:
# https://www.gpsvisualizer.com/calculators

rooms_lat: [38.74584282796673, 38.70213919994125, 38.74499852902329, 38.70144477298908, 38.71109888002856, 38.70824853381608, 38.751007598903335, 38.75894252695615, 38.746816666725785, 38.82631965777253, 38.71694216351822, 39.00386501446866, 38.72084577055854, 38.94581115726129, 38.711728978929635, 38.71308536425166, 38.71580738854452, 38.72775473777788, 38.70730938949486, 38.711248688937985, 38.97143166706441, 38.71715322999262, 38.72231541441824, 38.712737943557215, 38.726883479571555, 38.71062379871473, 38.795819430893125, 39.005962728530676, 38.7464481134842, 38.720637221426024, 38.75089940151426, 38.70530645655638, 38.690300201133766, 38.71248860445329, 38.71208442812792, 38.75765204552851, 38.71740795835646, 38.73086044505078, 38.71395646824652, 38.72385745438006, 38.72675852686154, 38.70914362687136, 38.710019718476204, 38.69055218940939, 38.71497145542667, 38.69463473046933, 38.69974647942219, 38.7084587440487, 38.72591566775218, 38.71470213658233, 38.87050144309989, 38.8146947

In [12]:
# cleanData['City_Lat'] = cities_lat
# cleanData['City_Long'] = cities_long
cleanData['Calc_Distance'] = calculated_distance

cleanData.head()

,Price,Room_Id,Coordinates,Room_Type,City,Country,Room_Lat,Room_Long,Selection,Calc_Distance
0,125,2998105,"[38.74584282796673, -9.152359763007405]",Entire home/apt,Lisbon,Portugal,38.745843,-9.152360,"Lisbon, Portugal",1.774650
1,500,3003712,"[38.70213919994125, -9.177363170810427]",Private room,Lisbon,Portugal,38.702139,-9.177363,"Lisbon, Portugal",2.476753
2,650,3008031,"[38.74499852902329, -9.168202490560926]",Entire home/apt,Lisbon,Portugal,38.744999,-9.168202,"Lisbon, Portugal",2.211431
3,500,3025693,"[38.70144477298908, -9.240587547103432]",Private room,Lisbon,Portugal,38.701445,-9.240588,"Lisbon, Portugal",5.644946
4,37,3063405,"[38.71109888002856, -9.148347820132365]",Entire home/apt,Lisbon,Portugal,38.711099,-9.148348,"Lisbon, Portugal",0.910974


In [13]:
cleanData= cleanData.drop(columns=["Coordinates"])
cleanData.head()

,Price,Room_Id,Room_Type,City,Country,Room_Lat,Room_Long,Selection,Calc_Distance
0,125,2998105,Entire home/apt,Lisbon,Portugal,38.745843,-9.152360,"Lisbon, Portugal",1.774650
1,500,3003712,Private room,Lisbon,Portugal,38.702139,-9.177363,"Lisbon, Portugal",2.476753
2,650,3008031,Entire home/apt,Lisbon,Portugal,38.744999,-9.168202,"Lisbon, Portugal",2.211431
3,500,3025693,Private room,Lisbon,Portugal,38.701445,-9.240588,"Lisbon, Portugal",5.644946
4,37,3063405,Entire home/apt,Lisbon,Portugal,38.711099,-9.148348,"Lisbon, Portugal",0.910974


In [22]:
# #Export city names to csv
# cleanData.to_csv('../data.csv', index=False)

# # Import cities file as DataFrame
# cleanData_df = pd.read_csv("../data.csv")
# cleanData_df.head()

In [ ]:
# this site confirmed the calculation is correct:
# https://www.gpsvisualizer.com/calculators

In [72]:
cleanData = cleanData.rename(index=str, columns={"Room_Lat":"Latitude",
                                                 "Room_Long":"Longitude", 
                                                 "Calc_Distance":"Distance"})

cleanData.head()

,Price,Room_Id,Coordinates,Room_Type,City,Country,Latitude,Longitude,Selection
0,125,2998105,"[38.74584282796673, -9.152359763007405]",Entire home/apt,Lisbon,Portugal,38.745843,-9.152360,"Lisbon, Portugal"
1,500,3003712,"[38.70213919994125, -9.177363170810427]",Private room,Lisbon,Portugal,38.702139,-9.177363,"Lisbon, Portugal"
2,650,3008031,"[38.74499852902329, -9.168202490560926]",Entire home/apt,Lisbon,Portugal,38.744999,-9.168202,"Lisbon, Portugal"
3,500,3025693,"[38.70144477298908, -9.240587547103432]",Private room,Lisbon,Portugal,38.701445,-9.240588,"Lisbon, Portugal"
4,37,3063405,"[38.71109888002856, -9.148347820132365]",Entire home/apt,Lisbon,Portugal,38.711099,-9.148348,"Lisbon, Portugal"


In [15]:
conn_str = "root:sinkybay@localhost/airbnb_data"
engine = create_engine(f'mysql://{conn_str}')

if not database_exists(engine.url):
    create_database(engine.url)

In [18]:
engine.table_names()

['listings']

In [17]:
metadata = MetaData(engine)

listings = Table('listings', metadata,
                 Column('Price', Integer),
                 Column('Room_Id', String(10)),
                 Column('Room_Type', String(25)),
                 Column('City', String(50)),
                 Column('Country', String(50)),
                 Column('Latitude', Float),
                 Column('Longitude', Float),
                 Column('Selection', String(100)),
                 Column('Distance', Float))

metadata.create_all()
for _t in metadata.tables:
    print("Table: ",_t)

Table:  listings


In [19]:
cleanData.to_sql(name="listings", con=engine, if_exists="append",index=False)

In [77]:
selection_count_listings= cleanData.groupby(['Selection']).count()

selection_sum= cleanData.groupby(['Selection']).sum()

selection_sum
# total_rooms= len(cleanData["Room_Id"].unique())
?total_price = cleanData["Price"].sum()

# selection = pd.DataFrame(cleanData["Selection"].unique())

# city_listings = selection["Room_Id"].count()
# city_rooms_perc = total_rooms/city_rooms
city_price = selection["Price"].sum()
# city_avg_price = total_price / city_price



# len(purchase_df["Item ID"].unique())

# selection_df = pd.DataFrame({"#Rooms":room_count,"Avg Price":avg_price/player_count*100})
# gender_demog_df["Percentage of players"] = gender_demog_df["Percentage of players"].map("{:.0f}".format)

# print("# of Rooms:", total_rooms)
# print("Total price:", total_price)
# print("# of rooms per city:", city_listings)
# print("% of rooms per city :", city_rooms_perc)
# print("city price sum:", city_price)
# print("Room avg price:", city_avg_price)
# selection.head()

# selection_df = pd.DataFrame({"#Rooms":room_count,"Avg Price":avg_price/player_count*100})
# gender_demog_df["Percentage of players"] = gender_demog_df["Percentage of players"].map("{:.0f}".format)

,Price,Room_Id,Latitude,Longitude
Selection,,,,
"Lisbon, Portugal",11925,547319015,3875.515354,-920.259888


In [66]:
selection_df = pd.DataFrame(cleanData["Selection"].unique())

selection_df["# of Listings"] = total_rooms
selection_df["# of listings per city"] = city_listings
selection_df

,0,# of Listings,# of listings per city
0,"Lisbon, Portugal",100,2
1,100,100,2


In [20]:
cityList = pd.read_sql_query('select distinct Selection from listings', con=engine)
cityList.head

<bound method NDFrame.head of           Selection
0  Lisbon, Portugal>

In [21]:
cities = Table('cities', metadata,
                 Column('Selection', String(100))
              )

metadata.create_all()
for _t in metadata.tables:
    print("Table: ",_t)

Table:  listings
Table:  cities


In [ ]:
cityList.to_sql(name="cities", con=engine, if_exists="append",index=False)